# 多GPU计算

本节中我们将展示如何使用多块GPU计算，例如，使用多块GPU训练同一个模型。正如所期望的那样，运行本节中的程序需要至少2块GPU。事实上，一台机器上安装多块GPU很常见，这是因为主板上通常会有多个PCIe插槽。如果正确安装了NVIDIA驱动，我们可以通过`nvidia-smi`命令来查看当前计算机上的全部GPU。

In [1]:
!nvidia-smi

Mon Feb 24 21:01:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla M60           On   | 00000000:00:1D.0 Off |                    0 |
| N/A   27C    P8    22W / 150W |      0MiB /  7618MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla M60           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   30C    P8    22W / 150W |      0MiB /  7618MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

[“自动并行计算”](auto-parallelism.ipynb)一节介绍过，大部分运算可以使用所有的CPU的全部计算资源，或者单块GPU的全部计算资源。但如果使用多块GPU训练模型，我们仍然需要实现相应的算法。这些算法中最常用的叫作数据并行。


## 数据并行

数据并行目前是深度学习里使用最广泛的将模型训练任务划分到多块GPU的方法。回忆一下我们在[“小批量随机梯度下降”](../chapter_optimization/minibatch-sgd.ipynb)一节中介绍的使用优化算法训练模型的过程。下面我们就以小批量随机梯度下降为例来介绍数据并行是如何工作的。

假设一台机器上有$k$块GPU。给定需要训练的模型，每块GPU及其相应的显存将分别独立维护一份完整的模型参数。在模型训练的任意一次迭代中，给定一个随机小批量，我们将该批量中的样本划分成$k$份并分给每块显卡的显存一份。然后，每块GPU将根据相应显存所分到的小批量子集和所维护的模型参数分别计算模型参数的本地梯度。接下来，我们把$k$块显卡的显存上的本地梯度相加，便得到当前的小批量随机梯度。之后，每块GPU都使用这个小批量随机梯度分别更新相应显存所维护的那一份完整的模型参数。图8.1描绘了使用2块GPU的数据并行下的小批量随机梯度的计算。

![使用2块GPU的数据并行下的小批量随机梯度的计算](../img/data-parallel.svg)

为了从零开始实现多GPU训练中的数据并行，让我们先导入需要的包或模块。

In [1]:
import d2lzh as d2l
import mxnet as mx
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

## 定义模型

我们使用[“卷积神经网络（LeNet）”](../chapter_convolutional-neural-networks/lenet.ipynb)一节里介绍的LeNet来作为本节的样例模型。这里的模型实现部分只用到了`NDArray`。

In [2]:
# 初始化模型参数
scale = 0.01
W1 = nd.random.normal(scale=scale, shape=(20, 1, 3, 3))
b1 = nd.zeros(shape=20)
W2 = nd.random.normal(scale=scale, shape=(50, 20, 5, 5))
b2 = nd.zeros(shape=50)
W3 = nd.random.normal(scale=scale, shape=(800, 128))
b3 = nd.zeros(shape=128)
W4 = nd.random.normal(scale=scale, shape=(128, 10))
b4 = nd.zeros(shape=10)
params = [W1, b1, W2, b2, W3, b3, W4, b4]

# 定义模型
def lenet(X, params):
    h1_conv = nd.Convolution(data=X, weight=params[0], bias=params[1],
                             kernel=(3, 3), num_filter=20)
    h1_activation = nd.relu(h1_conv)
    h1 = nd.Pooling(data=h1_activation, pool_type='avg', kernel=(2, 2),
                    stride=(2, 2))
    h2_conv = nd.Convolution(data=h1, weight=params[2], bias=params[3],
                             kernel=(5, 5), num_filter=50)
    h2_activation = nd.relu(h2_conv)
    h2 = nd.Pooling(data=h2_activation, pool_type='avg', kernel=(2, 2),
                    stride=(2, 2))
    h2 = nd.flatten(h2)
    h3_linear = nd.dot(h2, params[4]) + params[5]
    h3 = nd.relu(h3_linear)
    y_hat = nd.dot(h3, params[6]) + params[7]
    return y_hat

# 交叉熵损失函数
loss = gloss.SoftmaxCrossEntropyLoss()

## 多GPU之间同步数据

我们需要实现一些多GPU之间同步数据的辅助函数。下面的`get_params`函数将模型参数复制到某块显卡的显存并初始化梯度。

In [3]:
def get_params(params, ctx):
    new_params = [p.copyto(ctx) for p in params]
    for p in new_params:
        p.attach_grad()
    return new_params

尝试把模型参数`params`复制到`gpu(0)`上。

In [4]:
new_params = get_params(params, mx.gpu(0))
print('b1 weight:', new_params[1])
print('b1 grad:', new_params[1].grad)

b1 weight: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
<NDArray 20 @gpu(0)>
b1 grad: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
<NDArray 20 @gpu(0)>


给定分布在多块显卡的显存之间的数据。下面的`allreduce`函数可以把各块显卡的显存上的数据加起来，然后再广播到所有的显存上。

In [10]:
import numpy as np
def bit_product_sum(x, y):
    return sum([item[0] * item[1] for item in zip(x, y)])

def allreduce(data):
    for i in range(1, len(data)):
        tmp = data[i].copyto(data[0].context)
        x = data[0].asnumpy()
        y = tmp.asnumpy()
        assert x.shape == y.shape
        if x.ndim == 1:#只计算bias的grad向量的余弦距离。
            cos = bit_product_sum(x, y) / (np.sqrt(bit_product_sum(x, x)) * np.sqrt(bit_product_sum(y, y)))
            print(cos)
        for i in range(len(data[0])):
            data[0][:] += tmp
    for i in range(1, len(data)):
        data[0].copyto(data[i])
        
def ssp_allreduce(data):
    for i in range(1, len(data) // 2): #加的时候是一半的数据，分发的时候所有节点否分发
        data[0][:] += data[i].copyto(data[0].context)
    for i in range(1, len(data)):
        data[0].copyto(data[i])
        
def partial_allreduce(data):
    for i in range(1, len(data)):
        tmp = data[i].copyto(data[0].context)
        l = len(data[0]) // 2
        data[0][:l] += tmp[:l]
        if i < len(data) // 2:
            data[0][l:] += tmp[:l]
    data[0][l:] += data[0][l:]
#         for j in range(len(data[0])): #加的时候是一半的数据，分发的时候所有节点否分发
#             if(j < len(data[0]) // 2):
#                 data[0][j] += tmp[j]
#             elif(i < len(data) // 2):
#                 data[0][j] += tmp[j]   #后一半的参数只用一半的batchsize
#             else:
#                 pass
#     for i in range(len(data[0])):
#         if i >= len(data[0])// 2:
#             data[0][i] = data[0][i] * 2
    for i in range(1, len(data)):
        data[0].copyto(data[i])
    

简单测试一下`allreduce`函数。

In [11]:
train(num_gpus=2, batch_size=256, lr=0.2)

running on: [gpu(0), gpu(1)]
0.6270066624853188
0.375800393482649
0.3683323186189875
0.012304025234070615
0.21864679494113257
0.43763146199423053
0.22895182883467718
0.15126383649197325
0.2911170016637901
0.4542592248452909
0.5550275319771916
0.7019430464594684
0.2035858724639278
0.6130217086260753
0.5778098551350812
0.32671798415491354
0.9696115599566906
0.7949097933398012
-0.0181594870060467
0.36653036158647034
-0.22423850871136838
0.10104356656728555
0.053406545904972895
-0.15557698036244558
0.9856989091386835
0.973104650748628
0.9268395937703555
0.69690980452599
0.9733565452819621
0.8020597742872885
0.6055072360865549
0.12213908475829033
0.9912423747543821
0.9205521890120534
0.7616992316848378
-0.3859419122401858
0.9903954033318689
0.8523497537944944
0.9101272865862872
0.663728183922044
0.9961419104521523
0.9719548675601657
0.9131093767000753
-0.06584185381197631
0.9877697326613438
0.9555189754186717
0.7916097456585599
-0.13590433456905376
0.9964978962897636
0.9737687262713121
0.85

/home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


0.0314356939014446
nan
nan
nan
-0.19426968934931546
nan
nan
nan
0.22842534852603524
nan
nan
nan
0.08062240222729163
nan
nan
nan
0.4063390620259672
nan
nan
nan
0.4958847681148872
nan
nan
nan
-0.05778285894187085
nan
nan
nan
0.2033856460919176
nan
nan
nan
-0.5450213994142871
nan
nan
nan
0.6297699851367763
nan
nan
nan
0.4964044170898061
nan
nan
nan
0.5136295685766212
nan
nan
nan
0.42038938114244584
nan
nan
nan
-0.06379268242493367
nan
nan
nan
-0.055468099871107415
nan
nan
nan
-0.32991344023741187
nan
nan
nan
0.006600782103646881
nan
nan
nan
0.568084857836605
nan
nan
nan
-0.5244323410842048
nan
nan
nan
0.3642884747978928
nan
nan
nan
-0.051965852019359736
nan
nan
nan
0.0653376582181533
nan
nan
nan
0.318646811577896
nan
nan
nan
-0.1885604175486659
nan
nan
nan
-0.5040198693384808
nan
nan
nan
-0.08060239905623541
nan
nan
nan
0.6682340846656015
nan
nan
nan
-0.16889328846105656
nan
nan
nan
0.1483151334950351
nan
nan
nan
0.413140750962493
nan
nan
nan
0.47553440275722
nan
nan
nan
0.468638692539665

In [19]:
data = [nd.ones((4, 2), ctx=mx.gpu(i)) * (i + 1) for i in range(2)]
print('before allreduce:', data)
# partial_allreduce(data)
allreduce(data)
print('after allreduce:', data)

before allreduce: [
[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
<NDArray 4x2 @gpu(0)>, 
[[2. 2.]
 [2. 2.]
 [2. 2.]
 [2. 2.]]
<NDArray 4x2 @gpu(1)>]
reduce process:=======================
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
[0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998]
after allreduce: [
[[3. 3.]
 [3. 3.]
 [3. 3.]
 [3. 3.]]
<NDArray 4x2 @gpu(0)>, 
[[3. 3.]
 [3. 3.]
 [3. 3.]
 [3. 3.]]
<NDArray 4x2 @gpu(1)>]


给定一个批量的数据样本，下面的`split_and_load`函数可以将其划分并复制到各块显卡的显存上。

In [5]:
def split_and_load(data, ctx):
    n, k = data.shape[0], len(ctx)
    m = n // k  # 简单起见，假设可以整除
    assert m * k == n, '# examples is not divided by # devices.'
    return [data[i * m: (i + 1) * m].as_in_context(ctx[i]) for i in range(k)]

让我们试着用`split_and_load`函数将6个数据样本平均分给2块显卡的显存。

In [9]:
batch = nd.arange(24).reshape((6, 4))
ctx = [mx.gpu(0), mx.gpu(1)]
splitted = split_and_load(batch, ctx)
print('input: ', batch)
print('load into', ctx)
print('output:', splitted)

input:  
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]
 [16. 17. 18. 19.]
 [20. 21. 22. 23.]]
<NDArray 6x4 @cpu(0)>
load into [gpu(0), gpu(1)]
output: [
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
<NDArray 3x4 @gpu(0)>, 
[[12. 13. 14. 15.]
 [16. 17. 18. 19.]
 [20. 21. 22. 23.]]
<NDArray 3x4 @gpu(1)>]


## 单个小批量上的多GPU训练

现在我们可以实现单个小批量上的多GPU训练了。它的实现主要依据本节介绍的数据并行方法。我们将使用刚刚定义的多GPU之间同步数据的辅助函数`allreduce`和`split_and_load`。

In [9]:
def train_batch(X, y, gpu_params, ctx, lr):
    # 当ctx包含多块GPU及相应的显存时，将小批量数据样本划分并复制到各个显存上
    gpu_Xs, gpu_ys = split_and_load(X, ctx), split_and_load(y, ctx) 
    with autograd.record():  # 在各块GPU上分别计算损失
        ls = [loss(lenet(gpu_X, gpu_W), gpu_y)
              for gpu_X, gpu_y, gpu_W in zip(gpu_Xs, gpu_ys, gpu_params)]
    for l in ls:  # 在各块GPU上分别反向传播
        l.backward()
    # 把各块显卡的显存上的梯度加起来，然后广播到所有显存上 
#     print("grad number is:", len(gpu_params[0]))
    for i in range(len(gpu_params[0])):
        allreduce([gpu_params[c][i].grad for c in range(len(ctx))])
    for param in gpu_params:  # 在各块显卡的显存上分别更新模型参数
        d2l.sgd(param, lr, X.shape[0])  # 这里使用了完整批量大小

        
def ssp_train(X, y, gpu_params, ctx, lr):
    gpu_Xs, gpu_ys = split_and_load(X, ctx), split_and_load(y, ctx) 
    with autograd.record():  # 在各块GPU上分别计算损失
        ls = [loss(lenet(gpu_X, gpu_W), gpu_y)
              for gpu_X, gpu_y, gpu_W in zip(gpu_Xs, gpu_ys, gpu_params)]
    for l in ls:  # 在各块GPU上分别反向传播
        l.backward()
    # 把各块显卡的显存上的梯度加起来，然后广播到所有显存上
    for i in range(len(gpu_params[0])):
        ssp_allreduce([gpu_params[c][i].grad for c in range(len(ctx))])
    for param in gpu_params:  # 在各块显卡的显存上分别更新模型参数
        d2l.sgd(param, lr, X.shape[0] // 2)  # 这里使用了完整批量大小

def partial_train(X, y, gpu_params, ctx, lr):
    gpu_Xs, gpu_ys = split_and_load(X, ctx), split_and_load(y, ctx) 
    with autograd.record():  # 在各块GPU上分别计算损失
        ls = [loss(lenet(gpu_X, gpu_W), gpu_y)
              for gpu_X, gpu_y, gpu_W in zip(gpu_Xs, gpu_ys, gpu_params)]
    for l in ls:  # 在各块GPU上分别反向传播
        l.backward()

#     loss_curve.append(sum(sum(ls) / len(ls)) / len(sum(ls) / len(ls)))
    # 把各块显卡的显存上的梯度加起来，然后广播到所有显存上
    for i in range(len(gpu_params[0])):
        partial_allreduce([gpu_params[c][i].grad for c in range(len(ctx))])
    for param in gpu_params:  # 在各块显卡的显存上分别更新模型参数
        d2l.sgd(param, lr, X.shape[0])  # 这里使用了完整批量大小


## 定义训练函数

现在我们可以定义训练函数了。这里的训练函数和[“softmax回归的从零开始实现”](../chapter_deep-learning-basics/softmax-regression-scratch.ipynb)一节定义的训练函数`train_ch3`有所不同。值得强调的是，在这里我们需要依据数据并行将完整的模型参数复制到多块显卡的显存上，并在每次迭代时对单个小批量进行多GPU训练。

In [7]:
def train(num_gpus, batch_size, lr):
    train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
    ctx = [mx.gpu(i) for i in range(num_gpus)]
    print('running on:', ctx)
    # 将模型参数复制到num_gpus块显卡的显存上
    gpu_params = [get_params(params, c) for c in ctx]
    for epoch in range(1):
        start = time.time()
        for X, y in train_iter:
            # 对单个小批量进行多GPU训练
#             print("train")
            train_batch(X, y, gpu_params, ctx, lr)
            nd.waitall()
        train_time = time.time() - start

        def net(x):  # 在gpu(0)上验证模型
            return lenet(x, gpu_params[0])

        test_acc = d2l.evaluate_accuracy(test_iter, net, ctx[0])
        print('epoch %d, time %.1f sec, test acc %.2f'
              % (epoch + 1, train_time, test_acc))

## 多GPU训练实验

让我们先从单GPU训练开始。设批量大小为256，学习率为0.2。

In [22]:
train(num_gpus=4, batch_size=256, lr=0.2)

running on: [gpu(0), gpu(1), gpu(2), gpu(3)]
epoch 1, time 10.0 sec, test acc 0.10
epoch 2, time 8.2 sec, test acc 0.67
epoch 3, time 8.7 sec, test acc 0.77
epoch 4, time 8.7 sec, test acc 0.75
epoch 5, time 8.6 sec, test acc 0.79
epoch 6, time 8.5 sec, test acc 0.79
epoch 7, time 8.5 sec, test acc 0.81
epoch 8, time 7.8 sec, test acc 0.84
epoch 9, time 9.2 sec, test acc 0.82
epoch 10, time 8.9 sec, test acc 0.84


保持批量大小和学习率不变，将使用的GPU数量改为2。可以看到，测试精度的提升同上一个实验中的结果大体相当。因为有额外的通信开销，所以我们并没有看到训练时间的显著降低。因此，我们将在下一节实验计算更加复杂的模型。

In [19]:
train(num_gpus=4, batch_size=256, lr=0.2)

running on: [gpu(0), gpu(1), gpu(2), gpu(3)]
epoch 1, time 9.4 sec, test acc 0.10
epoch 2, time 7.9 sec, test acc 0.59
epoch 3, time 9.1 sec, test acc 0.65
epoch 4, time 9.5 sec, test acc 0.72
epoch 5, time 8.5 sec, test acc 0.80
epoch 6, time 8.8 sec, test acc 0.81
epoch 7, time 8.7 sec, test acc 0.74
epoch 8, time 7.8 sec, test acc 0.82
epoch 9, time 8.9 sec, test acc 0.84
epoch 10, time 8.4 sec, test acc 0.83


In [32]:
train(num_gpus=4, batch_size=256, lr=0.2)

running on: [gpu(0), gpu(1), gpu(2), gpu(3)]
epoch 1, time 14.3 sec, test acc 0.17
epoch 2, time 14.5 sec, test acc 0.62
epoch 3, time 11.5 sec, test acc 0.69
epoch 4, time 12.6 sec, test acc 0.77
epoch 5, time 12.7 sec, test acc 0.80
epoch 6, time 13.0 sec, test acc 0.75
epoch 7, time 13.2 sec, test acc 0.79
epoch 8, time 12.9 sec, test acc 0.82
epoch 9, time 11.4 sec, test acc 0.85
epoch 10, time 12.2 sec, test acc 0.81


In [34]:
train(num_gpus=4, batch_size=256, lr=0.2)

running on: [gpu(0), gpu(1), gpu(2), gpu(3)]
epoch 1, time 14.6 sec, test acc 0.10
epoch 2, time 13.0 sec, test acc 0.60
epoch 3, time 13.4 sec, test acc 0.75
epoch 4, time 12.1 sec, test acc 0.73
epoch 5, time 12.7 sec, test acc 0.76
epoch 6, time 12.1 sec, test acc 0.81
epoch 7, time 13.0 sec, test acc 0.73
epoch 8, time 10.9 sec, test acc 0.78
epoch 9, time 10.7 sec, test acc 0.81
epoch 10, time 13.3 sec, test acc 0.81


In [1]:
import matplotlib.pyplot as plt
loss_curve=[]
train(num_gpus=4, batch_size=256, lr=0.2)
plt.plot(loss_curve)

NameError: name 'train' is not defined

In [12]:
print(len(loss_curve[0]))

64


In [13]:
import numpy as np
x = np.array([1,2,3,4])
print(x.ndim)

1
